In [62]:
# General libs for file paths, data extraction, etc
from glob import glob
from pathlib import Path
from os.path import basename
import zipfile # To extract zipfiles
import urllib3 
urllib3.disable_warnings() # Disable warnings for data download via API

# CDS API
import cdsapi

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

# creds
import sys
import os
sys.path.append('/Users/max/Deep_Sky/')
from creds import CDS_UID, CDS_API_key

In [63]:
URL = 'https://cds.climate.copernicus.eu/api/v2'
os.chdir('/Users/max/Deep_Sky/GitHub/Deep_Sky_Data_Science/scrapers/')
DATADIR = '../data/copernicus_data/'


In [20]:

experiments = ['historical',  'ssp126', 'ssp245', 'ssp585']
#experiments_2 = ['ssp1_1_9', 'ssp370']

models = ['hadgem3_gc31_ll', 'ukesm1_0_ll']

In [17]:
# Define the pre-industrial period
pre_industrial_start = '1850'
pre_industrial_end = '1900'

c = cdsapi.Client(url=URL, key="{id}:{key}".format(id = CDS_UID, key = CDS_API_key))


In [21]:
for i in experiments[1:]:
    for j in models:
        c.retrieve(
            'projections-cmip6',
            {
                'format': 'zip',
                'temporal_resolution': 'monthly',
                'experiment': f'{i}',
                'level': 'single_levels',
                'variable': 'near_surface_air_temperature',
                'model': f'{j}',
                'date': '20242015-01-01/2100-12-31',
            },
            f'{DATADIR}/cmip6_agg_hadgem/cmip6_monthly_2015-2100_{i}_{j}.zip')

2024-04-15 14:21:37,649 INFO Welcome to the CDS
2024-04-15 14:21:37,650 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-04-15 14:21:37,805 INFO Request is queued
2024-04-15 14:21:38,948 INFO Request is running
2024-04-15 14:22:10,990 INFO Request is completed
2024-04-15 14:22:10,995 INFO Downloading https://download-0010-clone.copernicus-climate.eu/cache-compute-0010/cache/data5/adaptor.esgf_wps.retrieve-1713205308.9741213-27911-4-2bb6b965-c4cc-47fa-ac30-f29bb46b6b89.zip to ../data/copernicus_data//cmip6_agg_hadgem/cmip6_monthly_2015-2100_ssp126_hadgem3_gc31_ll.zip (47.7M)
2024-04-15 14:22:21,391 INFO Download rate 4.6M/s   
2024-04-15 14:22:21,654 INFO Welcome to the CDS
2024-04-15 14:22:21,655 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-04-15 14:22:21,813 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data6/adaptor.esgf_wps.retrieve-1713

In [48]:
# DOWNLOAD DATA FOR HISTORICAL PERIOD
for j in models:
    c.retrieve(
        'projections-cmip6',
        {
            'format': 'zip',
            'temporal_resolution': 'monthly',
            'experiment': 'historical',
            'level': 'single_levels',
            'variable': 'near_surface_air_temperature',
            'model': f'{j}',
            'date': '1850-01-01/2014-12-31',
        },
        f'{DATADIR}/cmip6_agg_hadgem/cmip6_monthly_1850-2014_historical_{j}.zip')

2024-04-15 16:45:40,865 INFO Welcome to the CDS
2024-04-15 16:45:40,866 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-04-15 16:45:41,243 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.esgf_wps.retrieve-1713179948.2828627-27293-14-dfee6b1e-16b1-4139-861c-c7bfa75571c4.zip to ../data/copernicus_data//cmip6_agg_hadgem/cmip6_monthly_1850-2014_historical_hadgem3_gc31_ll.zip (114.1M)
2024-04-15 16:46:33,405 INFO Download rate 2.2M/s  
2024-04-15 16:46:33,689 INFO Welcome to the CDS
2024-04-15 16:46:33,690 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/projections-cmip6
2024-04-15 16:46:33,952 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data5/adaptor.esgf_wps.retrieve-1713183374.335189-18798-11-fe4a9a50-949a-4a39-8e92-b5bcb1364c18.zip to ../data/copernicus_data//cmip6_agg_hadgem/cmip6_monthly_1850-2014_histori

In [65]:
cmip6_zip_paths = glob(f'{DATADIR}/*.zip')
for j in cmip6_zip_paths:
    with zipfile.ZipFile(j, 'r') as zip_ref:
        zip_ref.extractall(f'{DATADIR}')

In [66]:
cmip6_nc = list()
cmip6_nc_rel = glob(f'{DATADIR}/tas*.nc')
for i in cmip6_nc_rel:
    cmip6_nc.append(os.path.basename(i))

In [67]:
cmip6_nc


['tas_Amon_UKESM1-0-LL_historical_r13i1p1f2_gn_185001-194912_v20190406.nc',
 'tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_195001-201412_v20190624.nc',
 'tas_Amon_HadGEM3-GC31-LL_ssp126_r1i1p1f3_gn_20240116-21001216_v20200114.nc',
 'tas_Amon_UKESM1-0-LL_ssp245_r13i1p1f2_gn_20240116-21001216_v20190507.nc',
 'tas_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_195001-201412_v20190406.nc',
 'tas_Amon_UKESM1-0-LL_ssp245_r1i1p1f2_gn_20240116-21001216_v20190507.nc',
 'tas_Amon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_20240116-21001216_v20190908.nc',
 'tas_Amon_UKESM1-0-LL_ssp585_r1i1p1f2_gn_20240116-21001216_v20190507.nc',
 'tas_Amon_UKESM1-0-LL_historical_r1i1p1f2_gn_185001-194912_v20190406.nc',
 'tas_Amon_UKESM1-0-LL_historical_r13i1p1f2_gn_195001-201412_v20190406.nc',
 'tas_Amon_HadGEM3-GC31-LL_historical_r1i1p1f3_gn_185001-194912_v20190624.nc',
 'tas_Amon_HadGEM3-GC31-LL_ssp585_r1i1p1f3_gn_20240116-21001216_v20200114.nc',
 'tas_Amon_UKESM1-0-LL_ssp126_r1i1p1f2_gn_20240116-21001216_v20190503.nc']

In [45]:
ds = xr.open_dataset(f'{DATADIR}{cmip6_nc[0]}')
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 144, lon: 192, time: 1200)
Coordinates:
  * time       (time) object 1850-01-16 00:00:00 ... 1949-12-16 00:00:00
  * lat        (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon        (lon) float64 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-01-30T02:03:38Z
    ...                     ...
    variable_id:            tas
    variable_name:          tas
    variant_label:          r13i1p1f2
    license:                CMIP6 model data produced by the Met Office Hadle...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/d18023a2-bfcd-4c81-8f45-0a32995a7d47

In [60]:
def geog_agg(fn):
    ds = xr.open_dataset(f'{DATADIR}{fn}')
    exp = ds.attrs['experiment_id']
    mod = ds.attrs['source_id']
    da = ds['tas']
    weights = np.cos(np.deg2rad(da.lat))
    weights.name = "weights"
    da_weighted = da.weighted(weights)
    da_agg = da_weighted.mean(['lat', 'lon'])
    da_yr = da_agg.groupby('time.year').mean()
    da_yr = da_yr - 273.15
    da_yr = da_yr.assign_coords(model=mod)
    da_yr = da_yr.expand_dims('model')
    da_yr = da_yr.assign_coords(experiment=exp)
    da_yr = da_yr.expand_dims('experiment')
    da_yr.to_netcdf(path=f'{DATADIR}/cmip6_agg_hadgem/new/cmip6_agg_{exp}_{mod}_{str(da_yr.year[0].values)}.nc')


In [61]:
for i in cmip6_nc:
    try:
        geog_agg(i)
    except: print(f'{i} failed')